In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../comments_final.csv')
df.head()

,id,message,time,relative_time,author
0,5657094723,"Cara... muito bom, estou acompanhando. Obrigad...","Sexta-Feira, 24 de Dezembro de 2021 às 06:34",2 anos atrás,Leandro Pereira Gomes
1,5703851441,"Nando, tudo bem? Ainda realiza cursos como fiz...","Sábado, 22 de Janeiro de 2022 às 22:28",2 anos atrás,Leonardo Konarzewski
2,5710662240,"Fala Leonardo! Não, parei de dar os cursos em ...","Quinta-Feira, 27 de Janeiro de 2022 às 16:41",2 anos atrás,Nando Vieira
3,4939847379,"Nando, como entro em contato com você?","Quarta-Feira, 3 de Junho de 2020 às 20:40",3 anos atrás,Jhonatan Petronilho
4,4880700638,"Muito bom o artigo! Valeu, Nando!","Sábado, 18 de Abril de 2020 às 21:03",3 anos atrás,Messias Henrique Vieira Silva


In [3]:
from collections import Counter

In [4]:
# autor que mais comentou
Counter(df['author']).most_common(20)

[('Nando Vieira', 631),
 ('Walter Cruz', 31),
 ('Leonardo Faria', 21),
 ('Guest', 18),
 ('Leonardo Faria Coelho', 17),
 ('Carlos Brando', 15),
 ('Alexandre', 13),
 ('Cezinha Anjos', 12),
 ('Luiz Rocha', 12),
 ('Lucas Húngaro', 12),
 ('Thiago Antonius', 12),
 ('Lucas Caton', 11),
 ('Rodrigo Rosenfeld Rosas', 11),
 ('Ricardo Pacheco', 11),
 ('Willian Fernandes', 10),
 ('Roger', 10),
 ('Jeferson', 10),
 ('Leonardo Fontenelle', 10),
 ('Yuri Malheiros', 10),
 ('Yui', 9)]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from itertools import chain

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
stop_words = stopwords.words('portuguese')

In [7]:
count_vectorizer = CountVectorizer()
word_tokenizer = count_vectorizer.build_tokenizer()

In [8]:
def tokenize(text):
    try:
        tokens = word_tokenizer(text)
        tokens = [w for w in tokens if w not in stop_words]
        tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]
        return tokens
    except:
        return None

In [9]:
df['tokens'] = df['message'].apply(tokenize)

In [13]:
# palavras mais recorrentes
Counter(list(chain.from_iterable(df['tokens']))).most_common(20)   

[('Nando', 557),
 ('http', 350),
 ('fazer', 344),
 ('post', 317),
 ('bom', 311),
 ('Eu', 294),
 ('usar', 270),
 ('pra', 264),
 ('Muito', 254),
 ('bem', 244),
 ('problema', 223),
 ('pode', 217),
 ('Não', 204),
 ('aqui', 196),
 ('Parabéns', 190),
 ('exemplo', 186),
 ('ter', 174),
 ('sobre', 171),
 ('Rails', 167),
 ('código', 167)]

In [ ]:
## AUTORES QUE MAIS COMENTAM